In [1]:
from datetime import datetime
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow_datasets as tfds

epochs = 30
batch_size = 64
image_w, image_h = 32, 32

!rm -rf ./logs/malaria/*

In [2]:
(ds_train_raw, ds_test_raw), ds_info = tfds.load(
    "malaria",
    split=["train[:80%]", "train[80%:]"],
    shuffle_files=False,
    as_supervised=True,
    with_info=True,
)

n_classes = ds_info.features["label"].num_classes
n = ds_info.splits["train"].num_examples


def normalize_img(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = layers.Resizing(image_h, image_w)(image)
    image = tf.reshape(image, [-1])
    label = tf.one_hot(tf.cast(label, tf.int32), n_classes)
    label = tf.cast(label, tf.float32)
    return image, label


ds_train_normalized = ds_train_raw.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE
).cache()

ds_test_normalized = ds_test_raw.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE
).cache()


def prepare(ds, batch_size=batch_size):
    return ds.shuffle(n).batch(batch_size).prefetch(tf.data.AUTOTUNE)


dims = list(ds_train_normalized.take(1))[0][0].shape[0]

print("n: ", n, "n_classes: ", n_classes, "dims: ", dims)


def minmax_reducer(current, input):
    X, _ = input
    return (
        tf.reduce_min([current[0], X], axis=0),
        tf.reduce_max([current[0], X], axis=0),
    )


x0, _ = list(ds_train_normalized.take(1))[0]
min_train, max_train = ds_train_normalized.reduce((x0, x0), minmax_reducer)


n:  27558 n_classes:  2 dims:  3072


In [3]:
RandomFourierFeatures = keras.layers.experimental.RandomFourierFeatures

model_svm = keras.Sequential(
    [
        layers.Input(shape=(dims,)),
        RandomFourierFeatures(
            output_dim=2000, scale=10.0, kernel_initializer="gaussian"
        ),
        layers.Dense(units=n_classes),
    ]
)
model_svm.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.hinge,
    metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
)

modeldir = "./logs/malaria/linear-8192-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_svm.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(
            log_dir=modeldir + "/log",
            histogram_freq=1,
        )
    ],
)
model_svm.save(modeldir + "/model")


Epoch 1/30
345/345 [==============================] - 8s 13ms/step - loss: 0.7756 - acc: 0.6638 - val_loss: 0.7355 - val_acc: 0.6785
Epoch 2/30
345/345 [==============================] - 2s 6ms/step - loss: 0.6947 - acc: 0.7019 - val_loss: 0.7198 - val_acc: 0.6840
Epoch 3/30
345/345 [==============================] - 2s 6ms/step - loss: 0.6681 - acc: 0.7142 - val_loss: 0.7241 - val_acc: 0.6885
Epoch 4/30
345/345 [==============================] - 2s 6ms/step - loss: 0.6540 - acc: 0.7207 - val_loss: 0.7469 - val_acc: 0.6696
Epoch 5/30
345/345 [==============================] - 2s 6ms/step - loss: 0.6420 - acc: 0.7270 - val_loss: 0.7224 - val_acc: 0.6845
Epoch 6/30
345/345 [==============================] - 2s 6ms/step - loss: 0.6350 - acc: 0.7298 - val_loss: 0.7289 - val_acc: 0.6774
Epoch 7/30
345/345 [==============================] - 2s 7ms/step - loss: 0.6309 - acc: 0.7311 - val_loss: 0.7046 - val_acc: 0.6936
Epoch 8/30
345/345 [==============================] - 2s 6ms/step - loss: 0

INFO:tensorflow:Assets written to: ./logs/malaria/linear-8192-20230309-215525/model\assets


In [4]:
def gen_samples(psi, t=1000):
    return [
        list(
            ds_train_raw.shuffle(n)
            .take(psi)
            .map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(psi)
            .as_numpy_iterator()
        )[0][0]
        for _ in range(t)
    ]


def _tf_ann(X, samples, p=2, soft=True):
    m_dis = None
    for i in range(samples.shape[0]):
        i_sample = samples[i : i + 1, :]
        l_dis = tf.math.reduce_sum((X - i_sample) ** p, axis=1, keepdims=True) ** (
            1 / p
        )
        if m_dis is None:
            m_dis = l_dis
        else:
            m_dis = tf.concat([m_dis, l_dis], 1)

    if soft:
        feature_map = tf.nn.softmax(-m_dis, axis=0)
    else:
        feature_map = tf.one_hot(tf.math.argmax(-m_dis, axis=1), samples.shape[0])
    # l_dis_min = tf.math.reduce_sum(m_dis * feature_map, axis=0)
    return feature_map


In [5]:
class IsolationEncodingLayer(layers.Layer):
    def __init__(self, samples, p=2, soft=True, **kwargs):
        super(IsolationEncodingLayer, self).__init__(**kwargs)
        self.samples = samples
        self.p = p
        self.soft = soft

    def call(self, inputs):
        return _tf_ann(inputs, self.samples, self.p, self.soft)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "samples": self.samples,
                "p": self.p,
                "soft": self.soft,
            }
        )
        return config


def build_model(t_samples, p=2, soft=True):
    t = len(t_samples)
    if t <= 0:
        raise ValueError("t <= 0")
    _, dims = t_samples[0].shape

    inputs = keras.Input(name="inputs_x", shape=(dims,))
    lambdas = [
        IsolationEncodingLayer(t_samples[i], p=p, soft=soft, name="ann_{}".format(i))(
            inputs
        )
        for i in range(t)
    ]
    concatenated = layers.Concatenate(axis=1, name="concatenated")(lambdas)
    outputs = layers.Dense(units=n_classes, name="outputs_y")(concatenated)

    model = keras.Model(name="isolation_encoding", inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.hinge,
        metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
    )
    return model


In [6]:
class FlexibleIsolationEncodingLayer(layers.Layer):
    def __init__(self, samples, p=2, **kwargs):
        super(FlexibleIsolationEncodingLayer, self).__init__(**kwargs)
        self.samples = samples
        self.p = p

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.dimential_weights = self.add_weight(
            name="dimential_weights",
            shape=(
                1,
                self.samples.shape[0],
            ),
            initializer="ones",
            trainable=True,
        )
        super(FlexibleIsolationEncodingLayer, self).build(input_shape)

    def call(self, inputs):
        return _tf_ann(inputs, self.samples, self.p) * self.dimential_weights

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "samples": self.samples,
                "p": self.p,
            }
        )
        return config


def build_flex_model(t_samples, p=2):
    t = len(t_samples)
    if t <= 0:
        raise ValueError("t <= 0")
    _, dims = t_samples[0].shape

    inputs = keras.Input(name="inputs_x", shape=(dims,))
    lambdas = [
        FlexibleIsolationEncodingLayer(t_samples[i], p=p, name="ann_flex_{}".format(i))(
            inputs
        )
        for i in range(t)
    ]
    concatenated = layers.Concatenate(axis=1, name="concatenated")(lambdas)
    outputs = layers.Dense(units=n_classes, name="outputs_y")(concatenated)

    model = keras.Model(name="isolation_encoding", inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.hinge,
        metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
    )
    return model


In [7]:
t_samples = gen_samples(psi=20, t=50)


model_hard_20_50 = build_model(t_samples, soft=False)
modeldir = "./logs/malaria/hard-20x50-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_hard_20_50.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_hard_20_50.save(modeldir + "/model")


Epoch 1/30
345/345 [==============================] - 68s 157ms/step - loss: 0.8039 - acc: 0.6350 - val_loss: 0.7641 - val_acc: 0.6468
Epoch 2/30
345/345 [==============================] - 45s 131ms/step - loss: 0.7487 - acc: 0.6569 - val_loss: 0.7522 - val_acc: 0.6548
Epoch 3/30
345/345 [==============================] - 45s 131ms/step - loss: 0.7338 - acc: 0.6629 - val_loss: 0.7477 - val_acc: 0.6584
Epoch 4/30
345/345 [==============================] - 45s 131ms/step - loss: 0.7254 - acc: 0.6667 - val_loss: 0.7462 - val_acc: 0.6584
Epoch 5/30
345/345 [==============================] - 45s 131ms/step - loss: 0.7185 - acc: 0.6702 - val_loss: 0.7447 - val_acc: 0.6606
Epoch 6/30
345/345 [==============================] - 45s 132ms/step - loss: 0.7142 - acc: 0.6737 - val_loss: 0.7469 - val_acc: 0.6597
Epoch 7/30
345/345 [==============================] - 45s 131ms/step - loss: 0.7112 - acc: 0.6756 - val_loss: 0.7467 - val_acc: 0.6607
Epoch 8/30
345/345 [==============================] - 4

INFO:tensorflow:Assets written to: ./logs/malaria/hard-20x50-20230309-215805/model\assets


In [8]:
model_soft_20_50 = build_model(t_samples, soft=True)
modeldir = "./logs/malaria/soft-20x50-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_soft_20_50.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_soft_20_50.save(modeldir + "/model")


Epoch 1/30
345/345 [==============================] - 74s 167ms/step - loss: 0.8604 - acc: 0.6060 - val_loss: 0.7927 - val_acc: 0.6526
Epoch 2/30
345/345 [==============================] - 48s 139ms/step - loss: 0.7830 - acc: 0.6515 - val_loss: 0.7641 - val_acc: 0.6557
Epoch 3/30
345/345 [==============================] - 48s 138ms/step - loss: 0.7621 - acc: 0.6571 - val_loss: 0.7486 - val_acc: 0.6607
Epoch 4/30
345/345 [==============================] - 47s 136ms/step - loss: 0.7491 - acc: 0.6640 - val_loss: 0.7375 - val_acc: 0.6629
Epoch 5/30
345/345 [==============================] - 47s 137ms/step - loss: 0.7425 - acc: 0.6653 - val_loss: 0.7339 - val_acc: 0.6678
Epoch 6/30
345/345 [==============================] - 48s 139ms/step - loss: 0.7333 - acc: 0.6693 - val_loss: 0.7339 - val_acc: 0.6685
Epoch 7/30
345/345 [==============================] - 49s 141ms/step - loss: 0.7266 - acc: 0.6723 - val_loss: 0.7268 - val_acc: 0.6707
Epoch 8/30
345/345 [==============================] - 5

INFO:tensorflow:Assets written to: ./logs/malaria/soft-20x50-20230309-222224/model\assets


In [9]:
model_flex_20_50 = build_flex_model(t_samples)
# model_flex_20_50.summary()
modeldir = "./logs/malaria/flex-20x50-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_flex_20_50.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_flex_20_50.save(modeldir + "/model")


Epoch 1/30
345/345 [==============================] - 79s 176ms/step - loss: 0.8524 - acc: 0.6109 - val_loss: 0.7876 - val_acc: 0.6511
Epoch 2/30
345/345 [==============================] - 50s 146ms/step - loss: 0.7765 - acc: 0.6531 - val_loss: 0.7592 - val_acc: 0.6598
Epoch 3/30
345/345 [==============================] - 50s 146ms/step - loss: 0.7584 - acc: 0.6612 - val_loss: 0.7485 - val_acc: 0.6646
Epoch 4/30
345/345 [==============================] - 51s 147ms/step - loss: 0.7452 - acc: 0.6650 - val_loss: 0.7361 - val_acc: 0.6676
Epoch 5/30
345/345 [==============================] - 51s 148ms/step - loss: 0.7371 - acc: 0.6664 - val_loss: 0.7293 - val_acc: 0.6664
Epoch 6/30
345/345 [==============================] - 51s 148ms/step - loss: 0.7279 - acc: 0.6708 - val_loss: 0.7195 - val_acc: 0.6696
Epoch 7/30
345/345 [==============================] - 51s 147ms/step - loss: 0.7213 - acc: 0.6725 - val_loss: 0.7154 - val_acc: 0.6749
Epoch 8/30
345/345 [==============================] - 5

INFO:tensorflow:Assets written to: ./logs/malaria/flex-20x50-20230309-224828/model\assets


In [10]:
t_samples = gen_samples(psi=100, t=10)

model_hard_100_10 = build_model(t_samples, soft=False)
modeldir = "./logs/malaria/hard-100x10-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_hard_100_10.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_hard_100_10.save(modeldir + "/model")


Epoch 1/30
345/345 [==============================] - 59s 131ms/step - loss: 0.8968 - acc: 0.6161 - val_loss: 0.8151 - val_acc: 0.6321
Epoch 2/30
345/345 [==============================] - 37s 108ms/step - loss: 0.7864 - acc: 0.6374 - val_loss: 0.7774 - val_acc: 0.6353
Epoch 3/30
345/345 [==============================] - 39s 113ms/step - loss: 0.7628 - acc: 0.6427 - val_loss: 0.7681 - val_acc: 0.6410
Epoch 4/30
345/345 [==============================] - 40s 115ms/step - loss: 0.7526 - acc: 0.6482 - val_loss: 0.7633 - val_acc: 0.6444
Epoch 5/30
345/345 [==============================] - 40s 116ms/step - loss: 0.7460 - acc: 0.6516 - val_loss: 0.7604 - val_acc: 0.6455
Epoch 6/30
345/345 [==============================] - 39s 112ms/step - loss: 0.7408 - acc: 0.6546 - val_loss: 0.7591 - val_acc: 0.6455
Epoch 7/30
345/345 [==============================] - 39s 112ms/step - loss: 0.7372 - acc: 0.6570 - val_loss: 0.7580 - val_acc: 0.6462
Epoch 8/30
345/345 [==============================] - 3

INFO:tensorflow:Assets written to: ./logs/malaria/hard-100x10-20230309-231626/model\assets


In [11]:
model_soft_100_10 = build_model(t_samples, soft=True)
modeldir = "./logs/malaria/soft-100x10-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_soft_100_10.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_soft_100_10.save(modeldir + "/model")


Epoch 1/30
345/345 [==============================] - 75s 176ms/step - loss: 0.8601 - acc: 0.6118 - val_loss: 0.7969 - val_acc: 0.6528
Epoch 2/30
345/345 [==============================] - 52s 150ms/step - loss: 0.7839 - acc: 0.6514 - val_loss: 0.7610 - val_acc: 0.6618
Epoch 3/30
345/345 [==============================] - 51s 147ms/step - loss: 0.7645 - acc: 0.6600 - val_loss: 0.7526 - val_acc: 0.6635
Epoch 4/30
345/345 [==============================] - 51s 147ms/step - loss: 0.7516 - acc: 0.6653 - val_loss: 0.7403 - val_acc: 0.6687
Epoch 5/30
345/345 [==============================] - 49s 143ms/step - loss: 0.7406 - acc: 0.6672 - val_loss: 0.7296 - val_acc: 0.6680
Epoch 6/30
345/345 [==============================] - 49s 142ms/step - loss: 0.7333 - acc: 0.6682 - val_loss: 0.7242 - val_acc: 0.6731
Epoch 7/30
345/345 [==============================] - 50s 145ms/step - loss: 0.7272 - acc: 0.6709 - val_loss: 0.7227 - val_acc: 0.6700
Epoch 8/30
345/345 [==============================] - 4

INFO:tensorflow:Assets written to: ./logs/malaria/soft-100x10-20230309-233820/model\assets


In [12]:
model_flex_100_10 = build_flex_model(t_samples)
# model_flex_20_50.summary()
modeldir = "./logs/malaria/flex-100x10-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_flex_100_10.fit(
    prepare(ds_train_normalized),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=prepare(ds_test_normalized),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_flex_100_10.save(modeldir + "/model")


Epoch 1/30
345/345 [==============================] - 71s 166ms/step - loss: 0.8544 - acc: 0.6048 - val_loss: 0.7830 - val_acc: 0.6586
Epoch 2/30
345/345 [==============================] - 49s 143ms/step - loss: 0.7780 - acc: 0.6530 - val_loss: 0.7587 - val_acc: 0.6609
Epoch 3/30
345/345 [==============================] - 50s 144ms/step - loss: 0.7588 - acc: 0.6603 - val_loss: 0.7426 - val_acc: 0.6651
Epoch 4/30
345/345 [==============================] - 49s 142ms/step - loss: 0.7438 - acc: 0.6658 - val_loss: 0.7371 - val_acc: 0.6649
Epoch 5/30
345/345 [==============================] - 49s 142ms/step - loss: 0.7348 - acc: 0.6665 - val_loss: 0.7365 - val_acc: 0.6615
Epoch 6/30
345/345 [==============================] - 49s 143ms/step - loss: 0.7258 - acc: 0.6709 - val_loss: 0.7195 - val_acc: 0.6763
Epoch 7/30
345/345 [==============================] - 49s 143ms/step - loss: 0.7208 - acc: 0.6714 - val_loss: 0.7181 - val_acc: 0.6794
Epoch 8/30
345/345 [==============================] - 5

INFO:tensorflow:Assets written to: ./logs/malaria/flex-100x10-20230310-000421/model\assets
